In [15]:
import cv2
import numpy as np

DB_path = './data/VOC2007_trainval'

id = 'JPEGImages/000017.jpg'
img = cv2.imread(f'{DB_path}/{id}')

bnd = '185 62 279 199,90 78 403 336'
cls = '14,12'
bnd = list(map(str, bnd.split(',')))
cls = list(map(int, cls.split(',')))

for bbox in bnd:
    box = list(map(int, bbox.split(' ')))
    img = cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)

cv2.imshow('t', img)
cv2.waitKey()
cv2.destroyAllWindows()



In [189]:
import torch
import os
import cv2
import torch
import pandas as pd

DB_path = './data/VOC2007_trainval'
output_shape = (416, 416)

label_csv = pd.read_csv('2007_train.csv')
labels = torch.arange(20)
one_hot = torch.nn.functional.one_hot(labels)

for idx in range(2):
    img_path = os.path.join(f'{DB_path}', label_csv.iloc[idx, 0])
    image = cv2.resize(cv2.imread(img_path), output_shape)
    image = torch.from_numpy(image).permute(2, 0, 1).float()

    bboxs = label_csv.iloc[idx, 1]
    bboxs = list(map(str, bboxs.split(',')))
    bboxs = np.array(bboxs)
    bbox = []
    for bx in bboxs:
        bbox.append(list(map(int, bx.split(' '))))
    bbox = np.array(bbox)

    clss = label_csv.iloc[idx, 2]
    clss = list(map(int, clss.split(',')))
    clss = np.array(clss)
    cls = one_hot[clss]
    sample = {'image': img_path, 'Bboxs': bbox, 'Classes': clss}

    print(img_path, bbox, cls)


# labels = torch.arange(20).view(-1, 1)
# print(labels)
# one_hot = torch.zeros(20, 20).scatter_(1, labels, 1)
# print(one_hot)
#
# one_hot = torch.FloatTensor(20, 20).zero_()
# target = one_hot.scatter_(1, labels.data, 1)
# print(target)

./data/VOC2007_trainval\JPEGImages/000012.jpg [[156  97 351 270]] tensor([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
./data/VOC2007_trainval\JPEGImages/000017.jpg [[185  62 279 199]
 [ 90  78 403 336]] tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])


In [175]:
import torch
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader

anchors_wh = torch.tensor([[10, 13], [16, 30], [33, 23],
                           [30, 61], [62, 45], [59, 119],
                           [116, 90], [156, 198], [373, 326]]).float() / 416

DB_path = './data/VOC2007_trainval'
csv_file = '2007_train.csv'

class CustomDataset(Dataset):
    def __init__(self, num_classes, output_shape=(416, 416)):
        self.label_csv = pd.read_csv(f'{csv_file}')
        self.num_classes = num_classes
        self.output_shape = output_shape

    def __len__(self):
        return len(self.label_csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(f'{DB_path}', self.label_csv.iloc[idx, 0])
        image = cv2.resize(cv2.imread(img_path), output_shape)
        image = torch.from_numpy(image).permute(2, 0, 1).float()

        bboxes, classes = self.parse_y_features(idx)

        sample = {'image': img_path, 'Bboxs': bboxes, 'Classes': classes}

        return sample

    def parse_y_features(self, idx):
        bboxs = self.label_csv.iloc[idx, 1]
        bboxs = list(map(str, bboxs.split(',')))
        bboxs = np.array(bboxs)
        bbox = []
        for bx in bboxs:
            bbox.append(list(map(int, bx.split(' '))))
        bbox = np.array(bbox)

        labels = torch.arange(self.num_classes)
        one_hot = torch.nn.functional.one_hot(labels)
        clss = self.label_csv.iloc[idx, 2]
        clss = list(map(int, clss.split(',')))
        clss = np.array(clss)
        cls = one_hot[clss]

        return bbox, cls

    def preprocess_label_for_one_class(self, bbox, classes, grid_size, valid_anchor):
        y = torch.zeros((grid_size, grid_size, 3, 5 + self.num_classes))
        anchor_idx = self.find_best_anchor(bbox)

        num_boxes = classes.shape[0]
        for i in range(num_boxes):
            curr_class = torch.tensor(classes[i]).float()
            curr_box = torch.tensor(bbox[i]).float()
            curr_anchor = anchor_idx[i]

            # 앵커는 현재 그리드 사이즈에 맞는 것만 사용, 인덱스 3개로 조정
            if curr_anchor in valid_anchor:
                adjusted_anchor_idx = curr_anchor % 3

                # loss 계산을 위해 (xmin, ymin, xmax, ymax)를 (x, y, w, h)로 변환
                curr_box_xy = (curr_box[..., 0:2] + curr_box[..., 2:4]) / 2
                curr_box_wh = curr_box[..., 2:4] - curr_box[..., 0:2]

                # grid cell의 index
                grid_cell_xy = curr_box_xy // float(1 / grid_size)
                # gird[y][x][anchor] 형태 = (tx, ty, bw, bh, obj, class)
                index = torch.tensor([grid_cell_xy[1], grid_cell_xy[0], adjusted_anchor_idx]).int()
                update = torch.cat((curr_box_xy, curr_box_wh, torch.tensor([1.0]).float(), curr_class), dim=0)

                # gird cell 하나에 대한 anchor 3개
                y[index[0]][index[1]][index[2]] = update

        return y

    def find_best_anchor(self, bbox):

        box_wh = torch.from_numpy(bbox[..., 2:4] - bbox[..., 0:2]).float()
        box_wh = box_wh.unsqueeze(1).repeat(1, anchors_wh.shape[0], 1).float()  # anchors_wh.shape[0] = 9

        intersection = torch.min(box_wh[..., 0], anchors_wh[..., 0]) * torch.min(box_wh[..., 1], anchors_wh[..., 1])
        box_area = box_wh[..., 0] * box_wh[..., 1]
        anchor_area = anchors_wh[..., 0] * anchors_wh[..., 1]

        iou = intersection / (box_area + anchor_area - intersection)
        anchor_idx = torch.argmax(iou, dim=-1).int()

        return anchor_idx

dataset = CustomDataset(20)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)

In [215]:
anchors_wh = torch.tensor([[10, 13], [16, 30], [33, 23],
                           [30, 61], [62, 45], [59, 119],
                           [116, 90], [156, 198], [373, 326]]).float() / 416

label_csv = pd.read_csv(f'{csv_file}')

def parse_y_features(idx):
    bboxs = label_csv.iloc[idx, 1]
    bboxs = list(map(str, bboxs.split(',')))
    bboxs = np.array(bboxs)
    bbox = []
    for bx in bboxs:
        bbox.append(list(map(int, bx.split(' '))))
    bbox = np.array(bbox)

    labels = torch.arange(20)
    one_hot = torch.nn.functional.one_hot(labels)
    clss = label_csv.iloc[idx, 2]
    clss = list(map(int, clss.split(',')))
    clss = np.array(clss)
    cls = one_hot[clss]

    return bbox, cls

def preprocess_label_for_one_class(bbox, classes, grid_size, valid_anchor):
    y = torch.zeros((grid_size, grid_size, 3, 5 + 20))
    anchor_idx = find_best_anchor(bbox)

    num_boxes = classes.shape[0]
    for i in range(num_boxes):
        curr_class = torch.tensor(classes[i]).float()
        curr_box = torch.tensor(bbox[i]).float()
        curr_anchor = anchor_idx[i]

        # 앵커는 현재 그리드 사이즈에 맞는 것만 사용, 인덱스 3개로 조정
        if curr_anchor in valid_anchor:
            adjusted_anchor_idx = curr_anchor % 3

            # loss 계산을 위해 (xmin, ymin, xmax, ymax)를 (x, y, w, h)로 변환
            curr_box_xy = (curr_box[..., 0:2] + curr_box[..., 2:4]) / 2
            curr_box_wh = curr_box[..., 2:4] - curr_box[..., 0:2]

            # grid cell의 index
            grid_cell_xy = curr_box_xy // float(1 / grid_size)
            # gird[y][x][anchor] 형태 = (tx, ty, bw, bh, obj, class)
            index = torch.tensor([grid_cell_xy[1], grid_cell_xy[0], adjusted_anchor_idx]).int()
            update = torch.cat((curr_box_xy, curr_box_wh, torch.tensor([1.0]).float(), curr_class), dim=0)
            print(index)
            print(update[0])
            print(y.shape)
            # gird cell 하나에 대한 anchor 3개
            y[index[0]][index[1]][index[2]] = update


    return y

def find_best_anchor(bbox):
    box_wh = torch.from_numpy(bbox[..., 2:4] - bbox[..., 0:2]).float()
    box_wh = box_wh.unsqueeze(1).repeat(1, anchors_wh.shape[0], 1).float()  # anchors_wh.shape[0] = 9

    intersection = torch.min(box_wh[..., 0], anchors_wh[..., 0]) * torch.min(box_wh[..., 1], anchors_wh[..., 1])
    box_area = box_wh[..., 0] * box_wh[..., 1]
    anchor_area = anchors_wh[..., 0] * anchors_wh[..., 1]

    iou = intersection / (box_area + anchor_area - intersection)
    anchor_idx = torch.argmax(iou, dim=-1).int()

    return anchor_idx


for idx in range(2):
    img_path = os.path.join(f'{DB_path}', label_csv.iloc[idx, 0])
    image = cv2.resize(cv2.imread(img_path), output_shape)
    image = torch.from_numpy(image).permute(2, 0, 1).float()

    bboxes, classes = parse_y_features(idx)
    label = (
        preprocess_label_for_one_class(bboxes, classes, 52, torch.tensor([0, 1, 2])),
        preprocess_label_for_one_class(bboxes, classes, 26, torch.tensor([3, 4, 5])),
        preprocess_label_for_one_class(bboxes, classes, 13, torch.tensor([6, 7, 8]))
    )
    sample = {'image': img_path, 'Bboxs': bboxes, 'Classes': classes}

print(bboxes, classes)

# print(sample)

tensor([2385, 3295,    2], dtype=torch.int32)
tensor(253.5000)
torch.Size([13, 13, 3, 25])


c:\users\mjw27\anaconda3\envs\jw\lib\site-packages\ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


IndexError: index 2385 is out of bounds for dimension 0 with size 13